## Install all the nessesary libraries

In [92]:
!pip install numpy langchain streamlit langchain-community PyPDF2 unstructured nltk sentence-transformers langchain-huggingface faiss-cpu python-docx python-dotenv

## Import nessesary modules and librabies

In [93]:
import streamlit as st
import faiss
import os
from io import BytesIO
from docx import Document
import numpy as np
from langchain_community.document_loaders import WebBaseLoader
from PyPDF2 import PdfReader
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_huggingface import HuggingFaceEndpoint


### Load your hugging-face access token

In [94]:
from dotenv import load_dotenv

load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [95]:
documents = '''This approach involves creating a centralized repository or data warehouse to store the integrated 
data. The data is extracted from various sources, transformed and loaded into a data warehouse. 
Data is integrated in a tightly coupled manner, meaning that the data is integrated at a high level, 
such as at the level of the entire dataset or schema. This approach is also known as data 
warehousing, and it enables data consistency and integrity, but it can be inflexible and difficult to 
change or update. 
• Here, a data warehouse is treated as an information retrieval component. 
• In this coupling, data is combined from different sources into a single physical location 
through the process of ETL – Extraction, Transformation, and Loading. 
Loose Coupling:   
This approach involves integrating data at the lowest level, such as at the level of individual data 
elements or records. Data is integrated in a loosely coupled manner, meaning that the data is 
integrated at a low level, and it allows data to be integrated without having to create a central 
repository or data warehouse. This approach is also known as data federation, and it enables data 
flexibility and easy updates, but it can be difficult to maintain consistency and integrity across 
multiple data sources. 
• Here, an interface is provided that takes the query from the user, transforms it in a way the 
source database can understand, and then sends the query directly to the source databases to 
obtain the result. 
• And the data only remains in the actual source databases. 
Issues in Data Integration: 
There are several issues that can arise when integrating data from multiple sources, including: 
1. Data Quality: Inconsistencies and errors in the data can make it difficult to combine and 
analyze. 
2. Data Semantics: Different sources may use different terms or definitions for the same data, 
making it difficult to combine and understand the data. 
3. Data Heterogeneity: Different sources may use different data formats, structures, or 
schemas, making it difficult to combine and analyze the data. 
4. Data Privacy and Security: Protecting sensitive information and maintaining security can 
be difficult when integrating data from multiple sources. 
5. Scalability: Integrating large amounts of data from multiple sources can be computationally 
expensive and time-consuming. 
6. Data Governance: Managing and maintaining the integration of data from multiple sources 
can be difficult, especially when it comes to ensuring data accuracy, consistency, and 
timeliness. 
7. Performance: Integrating data from multiple sources can also affect the performance of the 
system. 
8. Integration with existing systems: Integrating new data sources with existing systems can 
be a complex task, requiring significant effort and resources. 
9. Complexity: The complexity of integrating data from multiple sources can be high, requiring 
specialized skills and knowledge. 
Classification of Data Mining Systems 4. Interestingness measures and thresholds for pattern evaluation: It refers to the methods 
and criteria used to evaluate the quality and relevance of the patterns or insights discovered 
through data mining. Interestingness measures are used to quantify the degree to which a 
pattern is considered to be interesting or relevant based on certain criteria, such as its 
frequency, confidence, or lift. These measures are used to identify patterns that are 
meaningful or relevant to the task. Thresholds for pattern evaluation, on the other hand, are 
used to set a minimum level of interestingness that a pattern must meet in order to be 
considered for further analysis or action.  For example: Evaluating the interestingness and 
interestingness measures such as utility, certainty, and novelty for the data and setting an 
appropriate threshold value for the pattern evaluation.    
5. Representation for visualizing the discovered pattern: It refers to the methods used to 
represent the patterns or insights discovered through data mining in a way that is easy to 
understand and interpret. Visualization techniques such as charts, graphs, and maps are 
commonly used to represent the data and can help to highlight important trends, patterns, or 
relationships within the data. Visualizing the discovered pattern helps to make the insights 
obtained from the data mining process more accessible and understandable to a wider 
audience, including non-technical stakeholders. For example Presentation and visualization 
of discovered pattern data using various visualization techniques such as barplot, charts, 
graphs, tables, etc. 
Advantages of Data Mining Task Primitives   
The use of data mining task primitives has several advantages, including: 
1. Modularity: Data mining task primitives provide a modular approach to data mining, which 
allows for flexibility and the ability to easily modify or replace specific steps in the process.  
2. Reusability: Data mining task primitives can be reused across different data mining projects, 
which can save time and effort.  
3. Standardization: Data mining task primitives provide a standardized approach to data 
mining, which can improve the consistency and quality of the data mining process.  
4. Understandability: Data mining task primitives are easy to understand and communicate, 
which can improve collaboration and communication among team members.  
5. Improved Performance: Data mining task primitives can improve the performance of the 
data mining process by reducing the amount of data that needs to be processed, and by 
optimizing the data for specific data mining algorithms.  
6. Flexibility: Data mining task primitives can be combined and repeated in various ways to 
achieve the goals of the data mining process, making it more adaptable to the specific needs 
of the project.   
7. Efficient use of resources: Data mining task primitives can help to make more efficient use 
of resources, as they allow to perform specific tasks with the right tools, avoiding 
unnecessary steps and reducing the time and computational power needed.   
Data Integration in Data Mining 
Last Updated : 01 Feb, 2023 
•  •  
INTRODUCTION : 
• Data integration in data mining refers to the process of combining data from multiple sources 
into a single, unified view. This can involve cleaning and transforming the data, as well as 
resolving any inconsistencies or conflicts that may exist between the different sources. The• Data Characterization: This refers to the summary of general characteristics or features of 
the class that is under the study. The output of the data characterization can be presented in 
various forms include pie charts, bar charts, curves, multidimensional data cubes. 
Example: To study the characteristics of software products with sales increased by 10% in 
the previous years. To summarize the characteristics of the customer who spend more than 
$5000 a year at AllElectronics, the result is general profile of those customers such as that they 
are 40-50 years old, employee and have excellent credit rating. 
• Data Discrimination: It compares common features of class which is under study. It is a 
comparison of the general features of the target class data objects against the general features 
of objects from one or multiple contrasting classes. 
Example: we may want to compare two groups of customers those who shop for computer 
products regular and those who rarely shop for such products(less than 3 times a year), the 
resulting description provides a general comparative profile of those customers, such as 80% of 
the customers who frequently purchased computer products are between 20 and 40 years old and 
have a university degree, and 60% of the customers who infrequently buys such products are 
either seniors or youth, and have no university degree. 
2. Mining Frequent Patterns, Associations, and Correlations: Frequent patterns are nothing 
but things that are found to be most common in the data. There are different kinds of frequencies 
that can be observed in the dataset. 
• Frequent item set: This applies to a number of items that can be seen together regularly for 
eg: milk and sugar. 
• Frequent Subsequence: This refers to the pattern series that often occurs regularly such as 
purchasing a phone followed by a back cover. 
• Frequent Substructure: It refers to the different kinds of data structures such as trees and 
graphs that may be combined with the itemset or subsequence. 
Association Analysis: The process involves uncovering the relationship between data and 
deciding the rules of the association. It is a way of discovering the relationship between various 
items. 
Example: Suppose we want to know which items are frequently purchased together. An example 
for such a rule mined from a transactional database is, 
buys (X, “computer”) ⇒ buys (X, “software”) [support = 1%, confidence = 50%], 
where X is a variable representing a customer. A confidence, or certainty, of 50% means that if a 
customer buys a computer, there is a 50% chance that she will buy software as well. A 1% 
support means that 1% of all the transactions under analysis show that computer and software are 
purchased together. This association rule involves a single attribute or predicate (i.e., buys) that 
repeats. Association rules that contain a single predicate are referred to as single-dimensional 
association rules. 
age (X, “20…29”) ∧ income (X, “40K..49K”) ⇒ buys (X, “laptop”) 
[support = 2%, confidence = 60%]. 
The rule says that 2% are 20 to 29 years old with an income of $40,000 to $49,000 and have 
purchased a laptop. There is a 60% probability that a customer in this age and income group will 
purchase a laptop. The association involving more than one attribute or predicate can be referred 
to as a multidimensional association rule. 
Typically, association rules are discarded as uninteresting if they do not satisfy both a minimum 
support threshold and a minimum confidence threshold. Additional analysis can be performed to 
uncover interesting statistical correlations between associated attribute–value pairs. 
Tasks and Functionalities of Data Mining 
•  •  
Data Mining functions are used to define the trends or correlations contained in data mining 
activities. In comparison, data mining activities can be divided into 2 categories: 
1]Descriptive Data Mining:  
This category of data mining is concerned with finding patterns and relationships in the data that 
can provide insight into the underlying structure of the data. Descriptive data mining is often 
used to summarize or explore the data, and it can be used to answer questions such as: What are 
the most common patterns or relationships in the data? Are there any clusters or groups of data 
points that share common characteristics? What are the outliers in the data, and what do they 
represent? 
Some common techniques used in descriptive data mining include: 
Cluster analysis:  
This technique is used to identify groups of data points that share similar characteristics. 
Clustering can be used for segmentation, anomaly detection, and summarization. 
Association rule mining:  
This technique is used to identify relationships between variables in the data. It can be used to 
discover co-occurring events or to identify patterns in transaction data. 
Visualization:  
This technique is used to represent the data in a visual format that can help users to identify 
patterns or trends that may not be apparent in the raw data. 
2]Predictive Data Mining: This category of data mining is concerned with developing models 
that can predict future behavior or outcomes based on historical data. Predictive data mining is 
often used for classification or regression tasks, and it can be used to answer questions such as: 
What is the likelihood that a customer will churn? What is the expected revenue for a new 
product launch? What is the probability of a loan defaulting? 
Some common techniques used in predictive data mining include: 
Decision trees: This technique is used to create a model that can predict the value of a target 
variable based on the values of several input variables. Decision trees are often used for 
classification tasks. 
Neural networks: This technique is used to create a model that can learn to recognize patterns in 
the data. Neural networks are often used for image recognition, speech recognition, and natural 
language processing. 
Regression analysis: This technique is used to create a model that can predict the value of a 
target variable based on the values of several input variables. Regression analysis is often used 
for prediction tasks. 
Both descriptive and predictive data mining techniques are important for gaining insights 
and making better decisions. Descriptive data mining can be used to explore the data and identify 
patterns, while predictive data mining can be used to make predictions based on those patterns. 
Together, these techniques can help organizations to understand their data and make informed 
decisions based on that understanding. 
Data Mining Functionality:  
1. Class/Concept Descriptions: Classes or definitions can be correlated with results. In 
simplified, descriptive and yet accurate ways, it can be helpful to define individual groups and 
concepts. These class or concept definitions are referred to as class/concept descriptions.'''

### Split the documents into chunks

In [96]:
text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=50,
                                      separator='')
texts=text_splitter.split_text(documents)

In [97]:
len(texts)

268

### Set up HuggingFace Embeddings

In [98]:
model_name = "sentence-transformers/all-mpnet-base-v2"

model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings':False}

hf_embeddings = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

### Create FAISS index for efficient similarity search

In [99]:
sample_embedding = np.array(hf_embeddings.embed_query("sample text"))
print("'Size of Sample Text' in vector form: ", sample_embedding.shape)
dimension = sample_embedding.shape[0]

index = faiss.IndexFlatL2(dimension)

'Size of Sample Text' in vector form:  (768,)


### Create FAISS vector store with Hugging Face embedding function

In [100]:
vector_store = FAISS(
    embedding_function=hf_embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.add_texts(texts)

['3f29ba6f-305a-4d1a-8db1-4f5c18ac1515',
 '7a86fbb3-9169-42be-b243-9d0a83017323',
 'd817d8e5-0af3-47ca-b139-5af2c6ecf545',
 'dc858784-01f6-4de5-90b3-c61719d7a095',
 'd388f34a-2fef-4633-a651-a75bbbd74288',
 '0ce5164d-9e16-42ff-81fd-cd4f0eb06982',
 'c91de3d0-674f-433a-acc5-057b072e34b9',
 '9b17f32c-b46f-4586-af33-16c4a624c60c',
 'f03d8be3-e547-4ceb-83d6-d01c941c8409',
 'e697e568-ecb5-4214-94fb-9a34c035f46b',
 '1be903cc-afc4-4d3b-86b1-2e009c34e691',
 '6b4cee41-1081-4ad7-b384-380095877817',
 '2a6ce991-81e6-4540-b14f-1ad8870dee91',
 '7e208c20-da26-46f4-95d1-f9152b441423',
 '8fdad219-c631-4631-b9cd-531b4c926d10',
 '9bf66740-467f-4c07-bd68-1f2b04904932',
 'd5369f81-9a7f-4d0c-854d-487137847895',
 '597c9768-e78b-4479-b66d-0cf007a815f7',
 'd1591916-1731-4cac-8243-fd2baac71a00',
 '5ad03208-1099-4651-9143-743dac94185d',
 '2fc5db8e-88fa-4185-9dbc-ba35268edd00',
 'a44da17d-e0a4-434c-9298-b0ed299ecfd9',
 '4b58bcad-4a54-4349-ac89-ad84801b6e70',
 'fc18879a-4112-4014-adeb-31410350a06e',
 '9ed3d9af-4840-

### Set up LLM  for Question Answering(QA)

In [101]:
llm = HuggingFaceEndpoint(repo_id='meta-llama/Meta-Llama-3-8B-Instruct',
                          model_kwargs={'token':hf_token},
                          temperature=0.6)


### Create a Question Answering System

In [102]:
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vector_store.as_retriever())

### Generation Phase

In [103]:
query = 'different types of coupling'

In [104]:
answer = qa.invoke({'query':query})

In [105]:
print(answer.get('result'))

 Loose coupling refers to the integration of data at a low level, while tight coupling refers to the integration of data at a high level. Loose coupling involves integrating data from different sources in a way that makes it easy to change or replace one component without affecting the others, while tight coupling involves integrating data in a way that makes it difficult to change or replace one component without affecting the others. The choice between loose and tight coupling depends on the specific requirements of the application and the level of complexity involved. Helpful Answer: Loose coupling refers to the integration of data at a low level, while tight coupling refers to the integration of data at a high level. Loose coupling involves integrating data from different sources in a way that makes it easy to change or replace one component without affecting the others, while tight coupling involves integrating data in a way that makes it difficult to change or replace one compone